In [1]:
#!/usr/bin/env python
# coding: utf-8

import os,sys
import numpy as np
import math
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
#from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
#from keras.applications.resnet50 import preprocess_input, decode_predictions
#from PIL import Image
import random
import keras
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.models import Sequential
import keras.backend as K
import gc
import time

D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\toolkit

In [2]:
#help(keras.applications.mobilenetv2)
help(keras.applications.mobilenetv2)

Help on module keras.applications.mobilenetv2 in keras.applications:

NAME
    keras.applications.mobilenetv2

DATA
    absolute_import = _Feature((2, 5, 0, 'alpha', 1), (3, 0, 0, 'alpha', 0...
    division = _Feature((2, 2, 0, 'alpha', 2), (3, 0, 0, 'alpha', 0), 8192...
    print_function = _Feature((2, 6, 0, 'alpha', 2), (3, 0, 0, 'alpha', 0)...

FILE
    d:\toolkits.win\anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\keras\applications\mobilenetv2.py




In [3]:
def file2array(path, delimiter):
    recordlist = []
    fp = open(path, 'r', encoding='utf-8')
    content = fp.read()     # content现在是一行字符串，该字符串包含文件所有内容
    fp.close()
    rowlist = content.splitlines()  # 按行转换为一维表，splitlines默认参数是‘\n’
    # 逐行遍历
    # 结果按分隔符分割为行向量
    recordlist = [row.split(delimiter) for row in rowlist if row.strip()]
    return np.array(recordlist)

In [4]:
#import numpy as np
import scipy.ndimage


def ndarray_zoom_scaling(label, new_h, new_w):
    """
    Implement scaling for ndarray with scipy.ndimage.zoom
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_zoom_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 2 2 2 2 3 3 3]
     [1 1 1 2 2 2 2 3 3 3]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [7 7 7 8 8 8 8 9 9 9]
     [7 7 7 8 8 8 8 9 9 9]]
    """
    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]
    if len(label.shape) == 2:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w], order=0)
    else:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w, 1], order=0)
    return label_new


def ndarray_nearest_neighbour_scaling(label, new_h, new_w):
    """
    Implement nearest neighbour scaling for ndarray
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_nearest_neighbour_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [7 7 7 7 8 8 8 9 9 9]
     [7 7 7 7 8 8 8 9 9 9]]
    """
#    if len(label.shape) == 2:
#        label_new = np.zeros([new_h, new_w], dtype=label.dtype)
#    else:
#        label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)
    label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)

    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]

    y_pos = np.arange(new_h)
    x_pos = np.arange(new_w)
    y_pos = np.floor(y_pos / scale_h).astype(np.int32)
    x_pos = np.floor(x_pos / scale_w).astype(np.int32)

    y_pos = y_pos.reshape(y_pos.shape[0], 1)
    y_pos = np.tile(y_pos, (1, new_w))
    x_pos = np.tile(x_pos, (new_h, 1))
    assert y_pos.shape == x_pos.shape

    label_new[:, :] = label[y_pos[:, :], x_pos[:, :]]
    return label_new


In [5]:
def DataSet(iFold,noise_tag,dup_tag,rand_tag):
    #
    train_path =("../Data_CNN_Indus/F%d/SECOM_samples_train_F%d") % (iFold, iFold)    
    test_path  =("../Data_CNN_Indus/F%d/SECOM_samples_test_F%d") % (iFold, iFold)
    #
    data_train = file2array(train_path, " ")
    nrow_train = data_train.shape[0]
    ncol_train = data_train.shape[1]
    #
    data_test  = file2array(test_path, " ")
    nrow_test  = data_test.shape[0]
    ncol_test  = data_test.shape[1]
    #
    if ncol_train != ncol_test:
        print('Numbers of features are not consistant.')
    if nrow_train + nrow_test != 1567:
        print('Numbers of samples are not consistant.')
    #
    train_path =("../Data_CNN_Indus/F%d/SECOM_labels_train_F%d") % (iFold, iFold)    
    test_path  =("../Data_CNN_Indus/F%d/SECOM_labels_test_F%d") % (iFold, iFold)
    #
    lab_train = file2array(train_path, " ")
    #
    lab_test  = file2array(test_path, " ")
    #
    num_feature = ncol_train
    side_len  = math.ceil(math.sqrt(num_feature))
    num_class = 2
    #
    mat_ind_path =("../Data_CNN_Indus/Mat_ind_F%d.txt") % (iFold)
    #
    mat_ind = file2array(mat_ind_path, "\t")
    
    if (int)(rand_tag):
        tmp_vec = np.zeros(num_feature)
        for index in range(num_feature):
            r = (int)(index/side_len)
            c = (int)(index%side_len)
            tmp_vec[index] = int(mat_ind[r,c])
        index = [i for i in range(num_feature)]
        random.shuffle(index)
        tmp_vec = tmp_vec[index]
        for index in range(num_feature):
            r = (int)(index/side_len)
            c = (int)(index%side_len)
            mat_ind[r,c] = int(tmp_vec[index])

    tmp_cnt_per_class = np.zeros(num_class)
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        tmp_cnt_per_class[(int)(tmp)] = tmp_cnt_per_class[(int)(tmp)] + 1
        
    tmp_max = tmp_cnt_per_class[0]
    tmp_ind = 0
    for index in range(num_class):
        if tmp_cnt_per_class[index] > tmp_max:
            tmp_max = tmp_cnt_per_class[index]
            tmp_ind = index
            
    num_dup = np.zeros(num_class)
    for index in range(num_class):
        if (int)(dup_tag):
            num_dup[index] = math.floor(tmp_cnt_per_class[tmp_ind]/tmp_cnt_per_class[index])
        else:
            num_dup[index] = 1
        
    nrow_train_dup = 0
    for index in range(num_class):
        nrow_train_dup = nrow_train_dup + tmp_cnt_per_class[index] * num_dup[index]
        
    nrow_train_dup = (int)(nrow_train_dup)
    
    X_train = np.empty((nrow_train_dup, side_len, side_len, 1), dtype=float)
    Y_train = np.empty((nrow_train_dup, num_class), dtype=float)

    sam_count = 0
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        nDup = num_dup[(int)(tmp)]
        nDup = (int)(nDup)
        for iSam in range(nDup):
            tmp_cnt = 0
            for i in range(side_len):
                for j in range(side_len):
                    if tmp_cnt < num_feature:
                        tmp_ind = mat_ind[i,j]
                        X_train[sam_count,i,j,0] = float(data_train[index,(int)(tmp_ind)])
                    else:
                        X_train[sam_count,i,j,0] = 0.5
                    tmp_cnt = tmp_cnt + 1
            sam_count = sam_count + 1

    sam_count = 0
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        nDup = num_dup[(int)(tmp)]
        nDup = (int)(nDup)
        for iSam in range(nDup):
            tmp_v = np.zeros(num_class)
            tmp_v[(int)(tmp)] = 1
            Y_train[sam_count] = tmp_v
            sam_count = sam_count + 1
    
    X_test  = np.empty((nrow_test, side_len, side_len, 1), dtype=float)
    Y_test  = np.empty((nrow_test, num_class), dtype=float)

    for index in range(nrow_test):
        tmp_cnt = 0
        for i in range(side_len):
            for j in range(side_len):
                if tmp_cnt < num_feature:
                    tmp_ind = mat_ind[i,j]
                    tmp_var = float(data_test[index,(int)(tmp_ind)])
                    #print(tmp_var)
                    #if tmp_var < 0:
                    #    tmp_var = 0
                    #if tmp_var > 1:
                    #    tmp_var = 1
                    X_test[index,i,j,0] = tmp_var
                else:
                    X_test[index,i,j,0] = 0.5
                tmp_cnt = tmp_cnt + 1
    
    for index in range(nrow_test):
        tmp = lab_test[index,0]
        tmp_v = np.zeros(num_class)
        tmp_v[(int)(tmp)] = 1
        Y_test[index] = tmp_v
        
    if (int)(noise_tag):
        noise = np.random.normal(loc=0,scale=1e-2,size=X_train.shape)
        output = X_train + noise
        X_train = np.concatenate((X_train, output), axis=0)
        Y_train = np.concatenate((Y_train, Y_train), axis=0)
    
    index = [i for i in range(len(X_train))]
    random.shuffle(index)
    X_train = X_train[index]
    Y_train = Y_train[index]
    
    index = [i for i in range(len(X_test))]
    random.shuffle(index)
    X_test = X_test[index]    
    Y_test = Y_test[index]
    
    X_train_new = np.empty((len(X_train), 224, 224, 3))
    #Y_train_new = Y_train
    
    for index in range(len(X_train)):
        tmp_new = ndarray_nearest_neighbour_scaling(X_train[index],224,224)
        for c in range(3):
            X_train_new[index,:,:,c] = tmp_new[:,:,0]
    
    X_test_new = np.empty((len(X_test), 224, 224, 3))
    #Y_test_new = Y_test
    
    for index in range(len(X_test)):
        tmp_new = ndarray_nearest_neighbour_scaling(X_test[index],224,224)
        for c in range(3):
            X_test_new[index,:,:,c] = tmp_new[:,:,0]
            
    del X_train, X_test
    gc.collect()

    return X_train_new,Y_train,X_test_new,Y_test

In [6]:
#import tensorflow as tf
def LeakyRelu(x, leak=0.01, name="LeakyRelu"):
   with tf.variable_scope(name):
     f1 = 0.5 * (1 + leak)
     f2 = 0.5 * (1 - leak)
     return f1 * x + f2 * tf.abs(x)

In [7]:
def neural(channel,height,width,classes):
    input_shape = (channel,height,width)
    if K.image_data_format() == "channels_last":#确认输入维度,就是channel是在开头，还是结尾
        input_shape = (height,width,channel)
    model = Sequential()#顺序模型（keras中包括顺序模型和函数式API两种方式）
    model.add(Conv2D(6,(3,3),padding="valid",activation=LeakyRelu,input_shape=input_shape,name="conv1"))
    #model.add(Conv2D(6,(3,3),padding="valid",input_shape=input_shape,name="conv1"))
    #model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name="pool1"))
    model.add(Conv2D(12,(3,3),padding="valid",activation=LeakyRelu,name="conv2",))
    #model.add(Conv2D(12,(3,3),padding="valid",name="conv2",))
    #model.add(LeakyReLU(alpha=0.01))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),name="pool2"))
    model.add(Flatten())
    #model.add(Dense(192,activation="relu",name="fc1"))
    model.add(Dense(classes,activation="softmax",name="fc2"))
    return model


In [8]:
def get_statistics(model,sam_data,lab_data):
    num_class = 2

    #
    test_rows = len(lab_data)
    test_prc = np.zeros(num_class)
    test_rec = np.zeros(num_class)
    test_Fva = np.zeros(num_class)
    test_BER = np.zeros(num_class)
    tmp_beta = 1
    yhat_out    = model.predict(sam_data, batch_size=32, verbose=1)
    yhat_label  = np.zeros(len(lab_data))
    true_labels = np.zeros(len(lab_data))
    for index in range(len(lab_data)):
        tmp_vec = lab_data[index]
        tmp_max = tmp_vec[0]
        tmp_ind = 0
        for i in range(num_class):
            if tmp_max < tmp_vec[i]:
                tmp_max = tmp_vec[i]
                tmp_ind = i
        true_labels[index] = tmp_ind
        #
        tmp_vec = yhat_out[index]
        tmp_max = tmp_vec[0]
        tmp_ind = 0
        for i in range(num_class):
            if tmp_max < tmp_vec[i]:
                tmp_max = tmp_vec[i]
                tmp_ind = i
        yhat_label[index] = tmp_ind

    test_N_TP = np.zeros(num_class)
    test_N_FP = np.zeros(num_class)
    test_N_TN = np.zeros(num_class)
    test_N_FN = np.zeros(num_class)
    tmp_count = np.zeros(num_class)
    for i in range(test_rows):
        cur_label = yhat_label[i]
        true_label = true_labels[i]
        for j in range(num_class):
            if j == cur_label:
                if j == true_label:
                    test_N_TP[j]=test_N_TP[j]+1
                    tmp_count[j] = tmp_count[j] + 1
                else:
                    test_N_FP[j]=test_N_FP[j]+1
            else:
                if j == true_label:
                    test_N_FN[j]=test_N_FN[j]+1
                    tmp_count[j] = tmp_count[j] + 1
                else:
                    test_N_TN[j]=test_N_TN[j]+1

    for i in range(num_class):
        if test_N_TP[i] > 0:
            test_prc[i] = test_N_TP[i] / (test_N_TP[i] + test_N_FP[i])
        else:
            test_prc[i] = 0

        test_rec[i]= test_N_TP[i] / (test_N_TP[i] + test_N_FN[i])
        test_Fva[i]=(1 + tmp_beta*tmp_beta)*test_rec[i]*test_prc[i]/\
                      (tmp_beta*tmp_beta*(test_rec[i]+test_prc[i]))
        test_BER[i]= test_N_FN[i] / (test_N_TP[i] + test_N_FN[i])
        
    print(tmp_count)
        
    return test_N_TP,test_N_FP,test_N_TN,test_N_FN,tmp_count
    

In [9]:
all_TP_train=[]
all_FP_train=[]
all_TN_train=[]
all_FN_train=[]
all_count_train=[]
all_TP_test=[]
all_FP_test=[]
all_TN_test=[]
all_FN_test=[]
all_count_test=[]
all_time_train=[]
all_time_predict_train=[]
all_time_predict_test=[]

nRun = 10
side_len = 22
num_class = 2
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for fold in range(nRun):
                iFold = fold + 1
                X_train,Y_train,X_test,Y_test = DataSet(iFold,noise_tag,dup_tag,ran_tag)
                print('X_train shape : ',X_train.shape)
                print('Y_train shape : ',Y_train.shape)
                print('X_test shape : ',X_test.shape)
                print('Y_test shape : ',Y_test.shape)
                #print(X_train[0])
                #print(Y_train[0,1])
                print(noise_tag,dup_tag,ran_tag,iFold)

                # # model
                model = tf.keras.applications.mobilenet.MobileNet(
                    input_shape=None, 
                    alpha=1.0, 
                    depth_multiplier=1, 
                    include_top=True, 
                    weights=None, 
                    input_tensor=None, 
                    pooling='max', 
                    classes=2
                )

                ##
                #model = neural(1,side_len,side_len,num_class)

                model.compile(optimizer=tf.train.AdamOptimizer(0.001),
                              loss='categorical_crossentropy',
                              metrics=['accuracy'])

                # # train
                start = time.clock()
                model.fit(X_train, Y_train, epochs=10, batch_size=16)
                all_time_train.append(time.clock()-start)

                # # evaluate
                #model.evaluate(X_test, Y_test, batch_size=32)

                # #
                #models.append(model)

                # # save
                #save_path =("./my_resnet_model.h5_F%d") % (iFold)
                #model.save(save_path)

                #
                X_train,Y_train,X_test,Y_test = DataSet(iFold,0,0,ran_tag)
                start = time.clock()
                train_N_TP,train_N_FP,train_N_TN,train_N_FN,train_count = get_statistics(model,X_train,Y_train)
                all_time_predict_train.append(time.clock()-start)
                start = time.clock()
                test_N_TP,test_N_FP,test_N_TN,test_N_FN,test_count = get_statistics(model,X_test,Y_test)
                all_time_predict_test.append(time.clock()-start)
                all_TP_train.append(train_N_TP)
                all_FP_train.append(train_N_FP)
                all_TN_train.append(train_N_TN)
                all_FN_train.append(train_N_FN)
                all_count_train.append(train_count)
                all_TP_test.append(test_N_TP)
                all_FP_test.append(test_N_FP)
                all_TN_test.append(test_N_TN)
                all_FN_test.append(test_N_FN)
                all_count_test.append(test_count)


X_train shape :  (1410, 224, 224, 3)
Y_train shape :  (1410, 2)
X_test shape :  (157, 224, 224, 3)
Y_test shape :  (157, 2)
0 0 0 1
Epoch 1/10
1410/1410 [==============================] - 25s 18ms/step - loss: 0.3376 - acc: 0.9014 4s - loss: 0.3
Epoch 2/10
1410/1410 [==============================] - 15s 11ms/step - loss: 0.2649 - acc: 0.9312 2s - loss: 0.26
Epoch 3/10
1410/1410 [==============================] - 15s 11ms/step - loss: 0.2358 - acc: 0.9376
Epoch 4/10
1410/1410 [==============================] - 15s 11ms/step - loss: 0.2179 - acc: 0.9383 1s - loss: 0.2193 - acc:
Epoch 5/10
1410/1410 [==============================] - 15s 11ms/step - loss: 0.2276 - acc: 0.9326
Epoch 6/10
1410/1410 [==============================] - 14s 10ms/step - loss: 0.1753 - acc: 0.9369
Epoch 7/10
1410/1410 [==============================] - 14s 10ms/step - loss: 0.1618 - acc: 0.9418
Epoch 8/10
1410/1410 [==============================] - 14s 10ms/step - loss: 0.1446 - acc: 0.9511
Epoch 9/10
1410/1410

D:\toolkits.win\Anaconda3-5.3.1\envs\tensorflow-gpu-1.10\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in double_scalars


X_train shape :  (1410, 224, 224, 3)
Y_train shape :  (1410, 2)
X_test shape :  (157, 224, 224, 3)
Y_test shape :  (157, 2)
0 0 0 2
Epoch 1/10
1410/1410 [==============================] - 21s 15ms/step - loss: 0.3046 - acc: 0.9262
Epoch 2/10
1410/1410 [==============================] - 15s 10ms/step - loss: 0.2481 - acc: 0.9305
Epoch 3/10
1410/1410 [==============================] - 15s 10ms/step - loss: 0.2335 - acc: 0.9340
Epoch 4/10
1410/1410 [==============================] - 15s 11ms/step - loss: 0.2075 - acc: 0.9340
Epoch 5/10
1410/1410 [==============================] - 16s 11ms/step - loss: 0.2029 - acc: 0.9383
Epoch 6/10
1410/1410 [==============================] - 16s 11ms/step - loss: 0.1664 - acc: 0.9468
Epoch 7/10
1410/1410 [==============================] - 16s 11ms/step - loss: 0.1788 - acc: 0.9348
Epoch 8/10
1410/1410 [==============================] - 16s 11ms/step - loss: 0.1241 - acc: 0.9539
Epoch 9/10
1410/1410 [==============================] - 16s 11ms/step - loss

1411/1411 [==============================] - 16s 11ms/step - loss: 0.1995 - acc: 0.9362
Epoch 6/10
1411/1411 [==============================] - 16s 11ms/step - loss: 0.1883 - acc: 0.9412
Epoch 7/10
1411/1411 [==============================] - 16s 11ms/step - loss: 0.1674 - acc: 0.9454 0s - loss: 0.1612 - acc: 
Epoch 8/10
1411/1411 [==============================] - 16s 11ms/step - loss: 0.1363 - acc: 0.9546
Epoch 9/10
1411/1411 [==============================] - 16s 11ms/step - loss: 0.1276 - acc: 0.9575
Epoch 10/10
1411/1411 [==============================] - 9s 7ms/step
[1317.   94.]
156/156 [==============================] - 1s 4ms/step
[146.  10.]
X_train shape :  (1411, 224, 224, 3)
Y_train shape :  (1411, 2)
X_test shape :  (156, 224, 224, 3)
Y_test shape :  (156, 2)
0 0 0 9
Epoch 1/10
1411/1411 [==============================] - 32s 22ms/step - loss: 0.3073 - acc: 0.9150
Epoch 2/10
1411/1411 [==============================] - 16s 11ms/step - loss: 0.2722 - acc: 0.9270
Epoch 3/10

1410/1410 [==============================] - 16s 11ms/step - loss: 0.1697 - acc: 0.9447
Epoch 10/10
1410/1410 [==============================] - 14s 10ms/step
[1317.   93.]
157/157 [==============================] - 1s 4ms/step
[146.  11.]
X_train shape :  (1410, 224, 224, 3)
Y_train shape :  (1410, 2)
X_test shape :  (157, 224, 224, 3)
Y_test shape :  (157, 2)
0 0 1 5
Epoch 1/10
1410/1410 [==============================] - 41s 29ms/step - loss: 0.2972 - acc: 0.9284
Epoch 2/10
1410/1410 [==============================] - 16s 11ms/step - loss: 0.2530 - acc: 0.9312
Epoch 3/10
1410/1410 [==============================] - 16s 11ms/step - loss: 0.2329 - acc: 0.9312
Epoch 4/10
1410/1410 [==============================] - 16s 11ms/step - loss: 0.2220 - acc: 0.9355
Epoch 5/10
1410/1410 [==============================] - 16s 11ms/step - loss: 0.1706 - acc: 0.9482 4s - 
Epoch 6/10
1410/1410 [==============================] - 16s 11ms/step - loss: 0.1918 - acc: 0.9390
Epoch 7/10
1410/1410 [======

...

157/157 [==============================] - 1s 4ms/step
[147.  10.]
X_train shape :  (2820, 224, 224, 3)
Y_train shape :  (2820, 2)
X_test shape :  (157, 224, 224, 3)
Y_test shape :  (157, 2)
1 0 1 3
Epoch 1/10
2820/2820 [==============================] - 225s 80ms/step - loss: 0.2724 - acc: 0.9280
Epoch 2/10
2820/2820 [==============================] - 35s 12ms/step - loss: 0.2085 - acc: 0.9415
Epoch 3/10
2820/2820 [==============================] - 35s 13ms/step - loss: 0.1705 - acc: 0.9454
Epoch 4/10
2820/2820 [==============================] - 35s 13ms/step - loss: 0.1610 - acc: 0.9511
Epoch 5/10
2820/2820 [==============================] - 35s 13ms/step - loss: 0.1226 - acc: 0.9564
Epoch 6/10
2820/2820 [==============================] - 35s 12ms/step - loss: 0.0965 - acc: 0.9674
Epoch 7/10
2820/2820 [==============================] - 35s 13ms/step - loss: 0.0862 - acc: 0.9667
Epoch 8/10
2820/2820 [==============================] - 35s 13ms/step - loss: 0.0822 - acc: 0.9702
Epoch 9/

2822/2822 [==============================] - 36s 13ms/step - loss: 0.1238 - acc: 0.9596
Epoch 6/10
2822/2822 [==============================] - 36s 13ms/step - loss: 0.1056 - acc: 0.9628
Epoch 7/10
2822/2822 [==============================] - 36s 13ms/step - loss: 0.0793 - acc: 0.9713
Epoch 8/10
2822/2822 [==============================] - 36s 13ms/step - loss: 0.0645 - acc: 0.9759
Epoch 9/10
2822/2822 [==============================] - 36s 13ms/step - loss: 0.0699 - acc: 0.9727
Epoch 10/10
1411/1411 [==============================] - 113s 80ms/step
[1317.   94.]
156/156 [==============================] - 1s 4ms/step
[146.  10.]
X_train shape :  (2822, 224, 224, 3)
Y_train shape :  (2822, 2)
X_test shape :  (156, 224, 224, 3)
Y_test shape :  (156, 2)
1 0 1 10
Epoch 1/10
2822/2822 [==============================] - 278s 99ms/step - loss: 0.2707 - acc: 0.9316
Epoch 2/10
2822/2822 [==============================] - 35s 12ms/step - loss: 0.2093 - acc: 0.9444
Epoch 3/10
2822/2822 [=========

1410/1410 [==============================] - 140s 99ms/step
[1317.   93.]
157/157 [==============================] - 1s 5ms/step
[146.  11.]
X_train shape :  (5238, 224, 224, 3)
Y_train shape :  (5238, 2)
X_test shape :  (157, 224, 224, 3)
Y_test shape :  (157, 2)
1 1 0 6
Epoch 1/10
5238/5238 [==============================] - 367s 70ms/step - loss: 0.3469 - acc: 0.8463
Epoch 2/10
5238/5238 [==============================] - 71s 14ms/step - loss: 0.1708 - acc: 0.9336
Epoch 3/10
5238/5238 [==============================] - 70s 13ms/step - loss: 0.1239 - acc: 0.9506
Epoch 4/10
5238/5238 [==============================] - 68s 13ms/step - loss: 0.0995 - acc: 0.9660
Epoch 5/10
5238/5238 [==============================] - 68s 13ms/step - loss: 0.0772 - acc: 0.9750
Epoch 6/10
5238/5238 [==============================] - 69s 13ms/step - loss: 0.0834 - acc: 0.9675
Epoch 7/10
5238/5238 [==============================] - 68s 13ms/step - loss: 0.0520 - acc: 0.9800
Epoch 8/10
5238/5238 [===========

5264/5264 [==============================] - 69s 13ms/step - loss: 0.1124 - acc: 0.9580
Epoch 4/10
5264/5264 [==============================] - 69s 13ms/step - loss: 0.0843 - acc: 0.9696
Epoch 5/10
5264/5264 [==============================] - 69s 13ms/step - loss: 0.0959 - acc: 0.9645
Epoch 6/10
5264/5264 [==============================] - 69s 13ms/step - loss: 0.0718 - acc: 0.9738
Epoch 7/10
5264/5264 [==============================] - 69s 13ms/step - loss: 0.0573 - acc: 0.9778
Epoch 8/10
5264/5264 [==============================] - 69s 13ms/step - loss: 0.0508 - acc: 0.9827
Epoch 9/10
5264/5264 [==============================] - 69s 13ms/step - loss: 0.0650 - acc: 0.9715
Epoch 10/10
1410/1410 [==============================] - 192s 136ms/step
[1316.   94.]
157/157 [==============================] - 1s 4ms/step
[147.  10.]
X_train shape :  (5264, 224, 224, 3)
Y_train shape :  (5264, 2)
X_test shape :  (157, 224, 224, 3)
Y_test shape :  (157, 2)
1 1 1 3
Epoch 1/10
5264/5264 [==========

5266/5266 [==============================] - 68s 13ms/step - loss: 0.0560 - acc: 0.9812
Epoch 10/10
1411/1411 [==============================] - 238s 168ms/step
[1317.   94.]
156/156 [==============================] - 1s 4ms/step
[146.  10.]
X_train shape :  (5266, 224, 224, 3)
Y_train shape :  (5266, 2)
X_test shape :  (156, 224, 224, 3)
Y_test shape :  (156, 2)
1 1 1 9
Epoch 1/10
5266/5266 [==============================] - 576s 109ms/step - loss: 0.3555 - acc: 0.8367
Epoch 2/10
5266/5266 [==============================] - 71s 14ms/step - loss: 0.1617 - acc: 0.9408
Epoch 3/10
5266/5266 [==============================] - 72s 14ms/step - loss: 0.1127 - acc: 0.9548
Epoch 4/10
5266/5266 [==============================] - 71s 14ms/step - loss: 0.0880 - acc: 0.9689
Epoch 5/10
5266/5266 [==============================] - 71s 14ms/step - loss: 0.0889 - acc: 0.9679
Epoch 6/10
5266/5266 [==============================] - 71s 14ms/step - loss: 0.0708 - acc: 0.9761
Epoch 7/10
5266/5266 [========

In [10]:
noise = np.random.normal(loc=0,scale=1e-2,size=X_train.shape)
output = X_train + noise
X_train_all = np.concatenate((X_train, output), axis=0)
print(noise)
print(noise.shape)
print(output.shape)
print(X_train.shape)
print(X_train_all.shape)
for i in range(len(X_train)):
    if (X_train[i]==X_train_all[i]).all()==False:
        print(i)
for i in range(len(X_train)):
    ind = len(X_train)+i
    if (output[i]==X_train_all[ind]).all()==False:
        print(ind)
        

[[[[-5.52400363e-03 -5.90399103e-03  1.91247519e-02]
   [ 4.90630304e-03 -1.07512101e-02 -5.10067384e-03]
   [ 5.10294778e-03  4.67972541e-03 -5.38270608e-03]
   ...
   [ 2.71822359e-03  1.47423216e-02  1.58259521e-02]
   [-4.13811058e-03  5.69349320e-03  1.22661872e-02]
   [-2.21102381e-02 -6.66349970e-03 -4.48259372e-05]]

  [[-3.29253754e-03 -5.70675888e-03 -6.17746265e-03]
   [-6.31940620e-03 -5.28882295e-04  2.03789663e-02]
   [ 2.32385582e-02 -1.31423128e-03 -1.57670878e-02]
   ...
   [-1.40420083e-02  2.27913906e-04 -6.59282836e-03]
   [ 9.46879750e-03  1.03342810e-02 -1.17941131e-02]
   [-2.25885354e-03 -1.05216882e-02 -4.11540841e-03]]

  [[-3.41034051e-03 -4.87531833e-03 -1.23816182e-02]
   [-2.90613332e-03  2.55900312e-03  3.66666176e-03]
   [ 2.17414261e-03 -9.11301339e-03  6.09534547e-03]
   ...
   [-1.67700533e-03  2.03040802e-02  8.40298011e-03]
   [-6.94594160e-03 -1.29050212e-02 -9.61107780e-03]
   [-9.86838318e-03 -8.17892152e-04 -2.59800277e-03]]

  ...

  [[ 1.83841

In [11]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_TP_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FP_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_TN_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FN_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
            for iRun in range(nRun):
                print(all_TP_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FP_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_TN_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_FN_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])

for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_count_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])
                print(all_count_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


[1314.   69.]
[25.  2.]
[  69. 1314.]
[ 2. 25.]
[1313.   38.]
[56.  3.]
[  38. 1313.]
[ 3. 56.]
[1276.   72.]
[22. 40.]
[  72. 1276.]
[40. 22.]
[1302.   54.]
[39. 15.]
[  54. 1302.]
[15. 39.]
[1316.   35.]
[58.  1.]
[  35. 1316.]
[ 1. 58.]
[1057.   89.]
[  4. 260.]
[  89. 1057.]
[260.   4.]
[1305.   78.]
[15. 12.]
[  78. 1305.]
[12. 15.]
[1315.   20.]
[74.  2.]
[  20. 1315.]
[ 2. 74.]
[1299.   64.]
[30. 18.]
[  64. 1299.]
[18. 30.]
[347.  94.]
[  0. 970.]
[ 94. 347.]
[970.   0.]
[145.   0.]
[10.  2.]
[  0. 145.]
[ 2. 10.]
[146.   0.]
[10.  1.]
[  0. 146.]
[ 1. 10.]
[143.   0.]
[10.  4.]
[  0. 143.]
[ 4. 10.]
[144.   2.]
[9. 2.]
[  2. 144.]
[2. 9.]
[145.   0.]
[11.  1.]
[  0. 145.]
[ 1. 11.]
[108.   5.]
[ 6. 38.]
[  5. 108.]
[38.  6.]
[138.   1.]
[10.  8.]
[  1. 138.]
[ 8. 10.]
[146.   0.]
[10.  0.]
[  0. 146.]
[ 0. 10.]
[141.   0.]
[10.  5.]
[  0. 141.]
[ 5. 10.]
[46.  6.]
[  4. 100.]
[ 6. 46.]
[100.   4.]
[1316.    0.]
[94.  0.]
[   0. 1316.]
[ 0. 94.]
[1316.    0.]
[94.  0.]
[   0. 1

In [12]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_time_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


162.8916595
164.94181790000002
167.84318010000004
172.48146210000004
169.98680089999993
171.74095140000009
174.1746587
176.7109108000002
179.84760630000005
180.84748029999992
181.1355481999999
183.85292860000027
187.7457546999999
184.75736619999998
188.44922110000016
190.63279650000004
195.48287610000034
199.09008459999995
200.44686479999973
203.47551380000004
346.18255879999924
344.0421245999996
349.2223646000002
351.75234700000055
356.14498510000067
356.2601895000007
362.95350599999983
365.9704858999994
371.53820389999964
375.0032840999993
381.85210729999926
383.4044417999994
390.77745909999976
393.04585450000013
404.14996449999853
411.87060869999914
416.8602952000001
417.4669135999993
427.0018497000001
432.76635309999983
464.43744030000016
469.44546470000023
476.39590940000016
478.1430055000001
487.929390700001
493.5829994000014
502.6768993000005
507.1807595999999
520.1061981999992
522.3557103999992
530.0024816000005
538.7249637999994
550.813032099999
557.1614640999978
561.588060599

In [13]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_time_predict_train[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


5.364831699999996
5.955427499999985
6.331547300000011
6.751699499999972
7.356577300000026
7.939250299999912
8.50451309999994
9.19883139999979
9.91418909999993
10.57601099999988
11.370773099999951
12.203954300000078
13.028779300000224
13.99210069999981
14.933256899999833
15.912774799999625
16.8567223
18.069391200000155
19.016620899999907
20.32903069999975
21.66594449999957
22.874642499999936
23.134908399999404
25.54164359999959
26.685093799999777
28.220642900000712
29.765488999999434
31.601767899999686
32.8211282000002
34.68890709999869
36.51617680000163
38.652187499999854
41.371064200000546
42.46286599999985
44.588662099999055
47.05170199999884
49.48587220000081
51.41737020000073
53.76499100000001
55.92352109999956
58.12843459999931
58.35166249999929
63.00924810000106
66.20310290000089
69.09720230000312
71.46561320000183
74.68188319999899
76.87507959999857
80.4083967000006
82.78882860000158
85.03095630000098
89.64061559999755
92.69855989999996
96.22561990000031
101.07867189999888
103.0

In [14]:
for noise_tag in range(2):
    for dup_tag in range(2):
        for ran_tag in range(2):
            for iRun in range(nRun):
                print(all_time_predict_test[noise_tag*2*2*nRun+dup_tag*2*nRun+ran_tag*nRun+iRun])


0.6097572999999841
0.5239713000000279
0.5338718000000426
0.5389923999999837
0.5331435000000511
0.5412974999999278
0.5348589000000175
0.6499100999999428
0.5352469000001747
0.5422435000000405
0.5614743999999519
0.5358932000003733
0.5501165999999102
0.5764451000000008
0.5536229999997886
0.5543096000001242
0.5585062000000107
0.5381335000001854
0.5705978999999388
0.558339100000012
0.557247599999755
0.553072599999723
0.5187064000001556
0.5720934000000852
0.579672899999423
0.567869200000132
0.5898195999998279
0.5683202000000165
0.5792763999997987
0.5616765000013402
0.572947300001033
0.5792763999997987
0.5668103999996674
0.5893373999988398
0.585732999999891
0.5904750000008789
0.6090569000007235
0.6031199999997625
0.5990514999994048
0.6109633000014583
0.5874750000002678
0.6072999999996682
0.6080910000000586
0.6186330999989877
0.6168644999997923
0.6204489000010653
0.618871599999693
0.5746925999992527
0.6341162999997323
0.6449180999989039
0.6332217999988643
0.643201400001999
0.6264363000009325
0.

In [15]:
# # restore
#model = tf.keras.models.load_model('my_resnet_model.h5')

# # test
#img_path = "../my_nn/dataset/test/medicine/IMG_20190717_135408_BURST91.jpg"
#img_path = "../my_nn/dataset/test/glue/IMG_20190717_135425_BURST91.jpg"

#img = image.load_img(img_path, target_size=(224, 224))

#plt.imshow(img)
#img = image.img_to_array(img)/ 255.0
#img = np.expand_dims(img, axis=0)  # 为batch添加第四维

#print(model.predict(img))
#np.argmax(model.predict(img))

#save_path =("./tmp_TP_EP%d") % (1)
#fl=open(save_path, 'w')
#lists=[line+"\n" for line in all_TP_train]
#fl.writelines(lists)